### Load BERT model for classification using Hugging Face

Example in this tutorial is obtained from various Hugging Face documentation pages.

**Detailed Step by Step Tutorial at: https://opendatascience.com/build-nlp-and-conversational-ai-apps-with-transformers-and-large-scale-pre-trained-language-models/**



In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

bert_model_name = 'bert-base-uncased'
distilbert_model_name = 'distilbert-base-uncased'

config = AutoConfig.from_pretrained(distilbert_model_name)
tokenizer = AutoTokenizer.from_pretrained(distilbert_model_name)
model = AutoModelForSequenceClassification.from_config(config)

### Load dataset using datasets library

In [2]:
from datasets import load_dataset
train_ds, test_ds = load_dataset('imdb', split=['train', 'test'])

Dataset imdb downloaded and prepared to /Users/chandra/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


### Or prepare data from scratch (this is similar to using your custom dataset)

In [6]:
#download dataset
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

from pathlib import Path
from sklearn.model_selection import train_test_split

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)
    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

# Taking sample dataset for faster processing
train_sample_size = 1000
eval_sample_size = 100
train_texts, train_labels = train_texts[:train_sample_size], train_labels[:train_sample_size]
test_texts, test_labels = test_texts[:eval_sample_size], test_labels[:eval_sample_size]

# further split train data into train and validation sets
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


--2021-03-21 00:55:14--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M   267KB/s    in 4m 20s  

2021-03-21 00:59:35 (316 KB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



### Obtain encodings using Tokenizer, i.e. convert text into embeddings used by the BERT model

In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

### Prepare dataset for training and validation

In [8]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

### Train the model using the Trainer class

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=20,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=40,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
40,0.042789
80,0.000060
120,0.000030
160,0.000019
200,0.000016


TrainOutput(global_step=226, training_loss=0.007597004417824535)

### Evaluate model on evaluate and test datasets

In [11]:
eval_results = trainer.evaluate()
print("Evaluation Results: ", eval_results)

Evaluation Results:  {'eval_loss': 9.199337000609376e-06, 'epoch': 2.0, 'total_flos': 370234423296000}


In [12]:
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results: ", test_results)

Test Results:  {'eval_loss': 9.20649017643882e-06, 'epoch': 2.0, 'total_flos': 370234423296000}
